Sin(x) Project 

Dagan Shoesmith and Michael Lyford

In [ ]:
# Import necessary libraries
import numpy as np 
import matplotlib.pyplot as plt


Fourier Series

Splines

Rational Function Interpolation